# Quantum Portfolio Optimization using QAOA

## A Complete Step-by-Step Guide

This notebook walks through the entire process of building a quantum portfolio optimizer:

1. **Problem Formulation** - Understanding the portfolio optimization problem
2. **QUBO Construction** - Converting to Quadratic Unconstrained Binary Optimization
3. **Ising Mapping** - Transforming QUBO to Ising Hamiltonian
4. **QAOA Implementation** - Building and running the quantum circuit
5. **Results Analysis** - Interpreting the optimization results

## Setup and Imports

In [ ]:
# Install dependencies if needed
# !pip install qiskit qiskit-algorithms qiskit-aer numpy pandas matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize

# Set up plotting
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [12, 6]

print("✅ All imports successful!")

---

## 1. Problem Formulation

### The Portfolio Optimization Problem

We want to select **k stocks** from **n available stocks** to maximize returns while minimizing risk.

### Mathematical Definition

**Decision Variables:**
$$x_i \in \{0, 1\}, \quad i = 1, 2, \ldots, n$$

Where:
- $x_i = 1$ if stock $i$ is selected
- $x_i = 0$ otherwise

**Objective Function (Risk-Return Trade-off):**
$$\min_x \left[ \lambda \cdot x^T \Sigma x - (1-\lambda) \cdot \mu^T x \right]$$

**Cardinality Constraint:**
$$\sum_{i=1}^n x_i = k$$

In [ ]:
# Sample stock data - 5 stocks for demonstration
stock_names = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']
n_stocks = len(stock_names)

# Expected annual returns (historical average)
expected_returns = np.array([0.15, 0.18, 0.16, 0.12, 0.14])

# Covariance matrix (annualized)
# This captures how stocks move together
covariance = np.array([
    [0.0625, 0.0200, 0.0180, 0.0150, 0.0160],  # RELIANCE
    [0.0200, 0.0484, 0.0350, 0.0120, 0.0130],  # TCS
    [0.0180, 0.0350, 0.0576, 0.0140, 0.0150],  # INFY
    [0.0150, 0.0120, 0.0140, 0.0400, 0.0300],  # HDFCBANK
    [0.0160, 0.0130, 0.0150, 0.0300, 0.0529],  # ICICIBANK
])

# Number of stocks to select
k = 3

# Risk aversion parameter (0.5 = balanced)
lambda_param = 0.5

print(f"📊 Problem Setup:")
print(f"   Stocks: {stock_names}")
print(f"   Select k = {k} stocks from n = {n_stocks}")
print(f"   Risk aversion λ = {lambda_param}")
print(f"\n📈 Expected Returns:")
for name, ret in zip(stock_names, expected_returns):
    print(f"   {name}: {ret*100:.1f}%")

In [ ]:
# Visualize the correlation matrix
volatility = np.sqrt(np.diag(covariance))
correlation = covariance / np.outer(volatility, volatility)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Correlation heatmap
im1 = axes[0].imshow(correlation, cmap='RdYlGn', vmin=-1, vmax=1)
axes[0].set_xticks(range(n_stocks))
axes[0].set_yticks(range(n_stocks))
axes[0].set_xticklabels(stock_names, rotation=45)
axes[0].set_yticklabels(stock_names)
axes[0].set_title('Correlation Matrix', fontsize=14, fontweight='bold')
plt.colorbar(im1, ax=axes[0])

# Risk vs Return scatter
axes[1].scatter(volatility * 100, expected_returns * 100, s=200, c='#6366f1', alpha=0.8)
for i, name in enumerate(stock_names):
    axes[1].annotate(name, (volatility[i]*100 + 0.5, expected_returns[i]*100), fontsize=10)
axes[1].set_xlabel('Volatility (%)', fontsize=12)
axes[1].set_ylabel('Expected Return (%)', fontsize=12)
axes[1].set_title('Risk-Return Profile', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 2. QUBO Formulation

### Converting to Unconstrained Form

We use the **penalty method** to embed the constraint into the objective:

$$\min_x \left[ \lambda \cdot x^T \Sigma x - (1-\lambda) \cdot \mu^T x + P \cdot \left(\sum_{i=1}^n x_i - k\right)^2 \right]$$

### QUBO Matrix Construction

This can be written as a QUBO problem: $\min_x x^T Q x$

**Expanding the penalty term:**
$$\left(\sum_i x_i - k\right)^2 = \sum_i x_i + 2\sum_{i<j} x_i x_j - 2k\sum_i x_i + k^2$$

Using $x_i^2 = x_i$ for binary variables:
$$= (1-2k)\sum_i x_i + 2\sum_{i<j} x_i x_j + k^2$$

**QUBO Matrix Elements:**
$$Q_{ii} = \lambda \cdot \sigma_{ii} - (1-\lambda) \cdot \mu_i + P \cdot (1 - 2k)$$
$$Q_{ij} = \lambda \cdot \sigma_{ij} + P \quad (i \neq j)$$

In [ ]:
def build_qubo_matrix(
    returns: np.ndarray,
    covariance: np.ndarray,
    k: int,
    lambda_param: float = 0.5,
    penalty: float = None
) -> np.ndarray:
    """
    Build the QUBO matrix for portfolio optimization.
    
    Args:
        returns: Expected returns vector (n,)
        covariance: Covariance matrix (n, n)
        k: Number of stocks to select
        lambda_param: Risk aversion [0, 1]
        penalty: Constraint penalty (auto if None)
    
    Returns:
        Q: QUBO matrix (n, n)
    """
    n = len(returns)
    
    # Auto-compute penalty if not provided
    if penalty is None:
        scale = max(np.max(np.abs(covariance)), np.max(np.abs(returns)))
        penalty = max(10.0, 2.0 * scale * n)
    
    # Initialize QUBO matrix
    Q = np.zeros((n, n))
    
    # Diagonal elements
    for i in range(n):
        Q[i, i] = (
            lambda_param * covariance[i, i]      # Risk
            - (1 - lambda_param) * returns[i]    # Return
            + penalty * (1 - 2 * k)              # Constraint
        )
    
    # Off-diagonal elements (upper triangular)
    for i in range(n):
        for j in range(i + 1, n):
            Q[i, j] = (
                lambda_param * covariance[i, j]  # Risk
                + penalty                         # Constraint
            )
    
    return Q, penalty

# Build QUBO matrix
Q, penalty = build_qubo_matrix(expected_returns, covariance, k, lambda_param)

print(f"📐 QUBO Matrix (penalty P = {penalty:.2f}):")
print()
print("    ", end="")
for name in stock_names:
    print(f"{name:>10}", end=" ")
print()
for i, name in enumerate(stock_names):
    print(f"{name:>4}", end=" ")
    for j in range(n_stocks):
        if j >= i:
            print(f"{Q[i,j]:>10.4f}", end=" ")
        else:
            print(f"{'':>10}", end=" ")
    print()

---

## 3. Ising Model Transformation

### Variable Transformation

Quantum computers naturally work with spin variables. We transform:

$$x_i = \frac{1 - z_i}{2}, \quad z_i \in \{-1, +1\}$$

### Ising Hamiltonian

The Ising Hamiltonian has the form:

$$H = \sum_i h_i Z_i + \sum_{i<j} J_{ij} Z_i Z_j + \text{offset}$$

Where:
- $h_i$ = linear (bias) coefficients
- $J_{ij}$ = coupling coefficients
- $Z_i$ = Pauli-Z operator on qubit $i$

In [ ]:
def qubo_to_ising(Q: np.ndarray) -> Tuple[np.ndarray, np.ndarray, float]:
    """
    Convert QUBO matrix to Ising coefficients.
    
    Args:
        Q: QUBO matrix (upper triangular or symmetric)
    
    Returns:
        h: Linear coefficients (n,)
        J: Coupling coefficients (n, n) - upper triangular
        offset: Constant energy offset
    """
    n = Q.shape[0]
    
    # Make symmetric
    Q_sym = (Q + Q.T) / 2
    
    # Compute linear terms
    h = np.zeros(n)
    for i in range(n):
        h[i] = -Q_sym[i, i] / 2
        for j in range(n):
            if j != i:
                h[i] -= Q_sym[i, j] / 4
    
    # Compute coupling terms
    J = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            J[i, j] = Q_sym[i, j] / 2
    
    # Compute offset
    offset = np.sum(Q_sym) / 4
    
    return h, J, offset

# Transform to Ising
h, J, offset = qubo_to_ising(Q)

print("🧲 Ising Model Coefficients:")
print()
print("Linear terms (h_i):")
for i, name in enumerate(stock_names):
    print(f"   h_{name} = {h[i]:.4f}")

print()
print("Coupling terms (J_ij):")
for i in range(n_stocks):
    for j in range(i + 1, n_stocks):
        if J[i, j] != 0:
            print(f"   J_{stock_names[i]},{stock_names[j]} = {J[i,j]:.4f}")

print(f"\nOffset: {offset:.4f}")

In [ ]:
# Build Hamiltonian string representation
def get_hamiltonian_str(h, J, stock_names):
    """Generate readable Hamiltonian expression."""
    terms = []
    n = len(h)
    
    for i in range(n):
        if abs(h[i]) > 1e-10:
            sign = "+" if h[i] >= 0 else ""
            terms.append(f"{sign}{h[i]:.3f}·Z_{stock_names[i]}")
    
    for i in range(n):
        for j in range(i + 1, n):
            if abs(J[i, j]) > 1e-10:
                sign = "+" if J[i, j] >= 0 else ""
                terms.append(f"{sign}{J[i,j]:.3f}·Z_{stock_names[i]}·Z_{stock_names[j]}")
    
    return "H = " + " ".join(terms)

print("\n📝 Ising Hamiltonian:")
print(get_hamiltonian_str(h, J, stock_names))

---

## 4. QAOA Implementation

### QAOA Circuit Structure

The QAOA circuit prepares the state:

$$|\psi(\gamma, \beta)\rangle = U_M(\beta_p) U_C(\gamma_p) \cdots U_M(\beta_1) U_C(\gamma_1) |+\rangle^{\otimes n}$$

### Cost Unitary
$$U_C(\gamma) = e^{-i\gamma H_C}$$

Implemented using:
- $e^{-i\gamma h_i Z_i}$ → RZ($2\gamma h_i$)
- $e^{-i\gamma J_{ij} Z_i Z_j}$ → CNOT-RZ-CNOT pattern

### Mixer Unitary
$$U_M(\beta) = e^{-i\beta H_M} = \prod_i e^{-i\beta X_i}$$

Implemented as RX($2\beta$) on each qubit.

In [ ]:
def create_qaoa_circuit(
    h: np.ndarray,
    J: np.ndarray,
    gamma: List[float],
    beta: List[float]
) -> QuantumCircuit:
    """
    Create QAOA circuit.
    
    Args:
        h: Linear coefficients
        J: Coupling coefficients
        gamma: Cost parameters (length p)
        beta: Mixer parameters (length p)
    
    Returns:
        QAOA quantum circuit
    """
    n = len(h)
    p = len(gamma)
    
    qc = QuantumCircuit(n)
    
    # Initial superposition
    qc.h(range(n))
    qc.barrier()
    
    for layer in range(p):
        # Cost unitary
        # Single-qubit Z rotations
        for i in range(n):
            if abs(h[i]) > 1e-10:
                qc.rz(2 * gamma[layer] * h[i], i)
        
        # Two-qubit ZZ interactions
        for i in range(n):
            for j in range(i + 1, n):
                if abs(J[i, j]) > 1e-10:
                    qc.cx(i, j)
                    qc.rz(2 * gamma[layer] * J[i, j], j)
                    qc.cx(i, j)
        
        qc.barrier()
        
        # Mixer unitary
        for i in range(n):
            qc.rx(2 * beta[layer], i)
        
        qc.barrier()
    
    # Measurement
    qc.measure_all()
    
    return qc

# Create a sample circuit with p=1
sample_circuit = create_qaoa_circuit(h, J, gamma=[0.5], beta=[0.5])
print("🔄 QAOA Circuit (p=1):")
print(sample_circuit.draw(output='text', fold=120))

In [ ]:
# Visualize the circuit
sample_circuit_viz = create_qaoa_circuit(h, J, gamma=[0.5], beta=[0.5])
sample_circuit_viz.draw(output='mpl', style='iqp', fold=40)

In [ ]:
def evaluate_expectation(
    params: np.ndarray,
    h: np.ndarray,
    J: np.ndarray,
    Q: np.ndarray,
    p: int,
    shots: int = 1024
) -> float:
    """
    Evaluate the expected cost for given parameters.
    
    Args:
        params: [gamma_1,...,gamma_p, beta_1,...,beta_p]
        h, J: Ising coefficients
        Q: QUBO matrix (for cost evaluation)
        p: QAOA depth
        shots: Number of measurements
    
    Returns:
        Expected cost value
    """
    gamma = params[:p]
    beta = params[p:]
    
    # Create circuit
    qc = create_qaoa_circuit(h, J, list(gamma), list(beta))
    
    # Run simulation
    sampler = StatevectorSampler()
    job = sampler.run([qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Compute expectation
    total_cost = 0.0
    total = 0
    
    for bitstring, count in counts.items():
        # Convert to binary array (reverse for qiskit ordering)
        x = np.array([int(b) for b in bitstring[::-1]])
        cost = x @ Q @ x
        total_cost += cost * count
        total += count
    
    return total_cost / total

# Test with random parameters
test_params = np.array([0.5, 0.5])  # p=1: [gamma, beta]
test_cost = evaluate_expectation(test_params, h, J, Q, p=1, shots=512)
print(f"Test expectation value: {test_cost:.4f}")

In [ ]:
def run_qaoa_optimization(
    h: np.ndarray,
    J: np.ndarray,
    Q: np.ndarray,
    p: int = 1,
    maxiter: int = 50,
    shots: int = 1024
) -> Dict:
    """
    Run complete QAOA optimization.
    
    Args:
        h, J: Ising coefficients
        Q: QUBO matrix
        p: QAOA depth
        maxiter: Maximum optimizer iterations
        shots: Measurement shots
    
    Returns:
        Optimization results dictionary
    """
    print(f"🚀 Starting QAOA Optimization (p={p})...")
    
    # Track costs
    costs = []
    
    def callback(params):
        cost = evaluate_expectation(params, h, J, Q, p, shots)
        costs.append(cost)
        if len(costs) % 10 == 0:
            print(f"   Iteration {len(costs)}: Cost = {cost:.4f}")
    
    # Initial parameters
    np.random.seed(42)
    initial_params = np.random.uniform(0, np.pi, 2 * p)
    
    # Optimize
    result = minimize(
        lambda params: evaluate_expectation(params, h, J, Q, p, shots),
        initial_params,
        method='COBYLA',
        options={'maxiter': maxiter, 'rhobeg': 0.5},
        callback=callback
    )
    
    # Get final state
    gamma_opt = result.x[:p]
    beta_opt = result.x[p:]
    
    final_qc = create_qaoa_circuit(h, J, list(gamma_opt), list(beta_opt))
    sampler = StatevectorSampler()
    job = sampler.run([final_qc], shots=shots * 4)
    final_counts = job.result()[0].data.meas.get_counts()
    
    # Find best solution
    best_bitstring = None
    best_cost = float('inf')
    
    for bitstring, count in final_counts.items():
        x = np.array([int(b) for b in bitstring[::-1]])
        cost = x @ Q @ x
        if cost < best_cost:
            best_cost = cost
            best_bitstring = bitstring[::-1]
    
    print(f"\n✅ Optimization Complete!")
    print(f"   Optimal cost: {best_cost:.4f}")
    print(f"   Best bitstring: {best_bitstring}")
    
    return {
        'optimal_params': result.x,
        'optimal_cost': best_cost,
        'best_bitstring': best_bitstring,
        'final_counts': final_counts,
        'cost_history': costs,
        'iterations': len(costs)
    }

# Run optimization
results = run_qaoa_optimization(h, J, Q, p=1, maxiter=30, shots=512)

---

## 5. Results Analysis

In [ ]:
# Visualize optimization convergence
plt.figure(figsize=(10, 5))
plt.plot(results['cost_history'], 'b-', linewidth=2, alpha=0.7)
plt.scatter(range(len(results['cost_history'])), results['cost_history'], c='#6366f1', s=50)
plt.axhline(y=results['optimal_cost'], color='#10b981', linestyle='--', label=f'Optimal: {results["optimal_cost"]:.4f}')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Expected Cost', fontsize=12)
plt.title('QAOA Optimization Convergence', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Visualize final state distribution
counts = results['final_counts']

# Sort by count
sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10])

fig, ax = plt.subplots(figsize=(12, 5))

# Highlight the best solution
colors = ['#10b981' if bs[::-1] == results['best_bitstring'] else '#6366f1' 
          for bs in sorted_counts.keys()]

bars = ax.bar(range(len(sorted_counts)), list(sorted_counts.values()), color=colors)
ax.set_xticks(range(len(sorted_counts)))
ax.set_xticklabels([bs[::-1] for bs in sorted_counts.keys()], rotation=45, ha='right')
ax.set_xlabel('Bitstring (Stock Selection)', fontsize=12)
ax.set_ylabel('Measurement Count', fontsize=12)
ax.set_title('QAOA Output Distribution', fontsize=14, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#10b981', label='Optimal Solution'),
                   Patch(facecolor='#6366f1', label='Other Solutions')]
ax.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

In [ ]:
# Interpret the solution
best_x = np.array([int(b) for b in results['best_bitstring']])
selected_indices = np.where(best_x == 1)[0]
selected_stocks = [stock_names[i] for i in selected_indices]

print("\n🎯 OPTIMAL PORTFOLIO")
print("=" * 50)
print(f"\n📊 Selected Stocks ({len(selected_stocks)} stocks):")
for i, stock in enumerate(selected_stocks):
    idx = stock_names.index(stock)
    print(f"   {i+1}. {stock} - Return: {expected_returns[idx]*100:.1f}%, Vol: {np.sqrt(covariance[idx,idx])*100:.1f}%")

# Calculate portfolio metrics (equal weighted)
n_selected = len(selected_indices)
weights = np.zeros(n_stocks)
weights[selected_indices] = 1.0 / n_selected

portfolio_return = np.dot(weights, expected_returns)
portfolio_variance = weights @ covariance @ weights
portfolio_risk = np.sqrt(portfolio_variance)
sharpe_ratio = portfolio_return / portfolio_risk

print(f"\n📈 Portfolio Metrics:")
print(f"   Expected Return: {portfolio_return*100:.2f}%")
print(f"   Portfolio Risk:  {portfolio_risk*100:.2f}%")
print(f"   Sharpe Ratio:    {sharpe_ratio:.3f}")

In [ ]:
# Compare with all valid portfolios
from itertools import combinations

all_portfolios = []
for combo in combinations(range(n_stocks), k):
    w = np.zeros(n_stocks)
    w[list(combo)] = 1.0 / k
    
    ret = np.dot(w, expected_returns)
    var = w @ covariance @ w
    risk = np.sqrt(var)
    sharpe = ret / risk
    
    names = [stock_names[i] for i in combo]
    all_portfolios.append({
        'stocks': names,
        'return': ret,
        'risk': risk,
        'sharpe': sharpe
    })

# Sort by Sharpe ratio
all_portfolios.sort(key=lambda x: x['sharpe'], reverse=True)

print("\n📊 All Possible Portfolios (sorted by Sharpe ratio):")
print("=" * 70)
print(f"{'Rank':<6}{'Stocks':<30}{'Return':>10}{'Risk':>10}{'Sharpe':>10}")
print("-" * 70)
for i, p in enumerate(all_portfolios, 1):
    stocks_str = ', '.join(p['stocks'])
    marker = "⭐" if set(p['stocks']) == set(selected_stocks) else "  "
    print(f"{marker}{i:<4}{stocks_str:<30}{p['return']*100:>9.2f}%{p['risk']*100:>9.2f}%{p['sharpe']:>10.3f}")

# Find QAOA result rank
qaoa_rank = next(i for i, p in enumerate(all_portfolios, 1) if set(p['stocks']) == set(selected_stocks))
print(f"\n🏆 QAOA found portfolio ranked #{qaoa_rank} out of {len(all_portfolios)} possible portfolios")

---

## Summary

In this notebook, we:

1. ✅ **Formulated** the portfolio optimization problem mathematically
2. ✅ **Constructed** the QUBO matrix with penalty for cardinality constraint
3. ✅ **Transformed** QUBO to Ising Hamiltonian
4. ✅ **Implemented** QAOA circuit using Qiskit
5. ✅ **Optimized** and found the optimal portfolio
6. ✅ **Validated** results against all possible portfolios

### Key Takeaways

- QAOA successfully finds optimal or near-optimal portfolios
- The penalty method effectively enforces the cardinality constraint
- Circuit depth `p` and iteration count affect solution quality
- This approach scales to larger problems on real quantum hardware